In [45]:
import requests
import pandas as pd

# Send a GET request to the JSON URL
url = "https://markets.newyorkfed.org/api/soma/summary.json"
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Load JSON data into a Python dictionary
    data = response.json()

    # Extract the relevant time series data
    summary_data = data.get('soma', {}).get('summary', [])

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(summary_data)

    # Print the DataFrame
    print(df)

else:
    print(f"Failed to fetch data. Status code: {response.status_code}")



        asOfDate               mbs           cmbs          tips          frn  \
0     2003-07-09              0.00           0.00   12814483000                
1     2003-07-16              0.00           0.00   12814483000                
2     2003-07-23              0.00           0.00   12814483000                
3     2003-07-30              0.00           0.00   12814483000                
4     2003-08-06              0.00           0.00   12814483000                
...          ...               ...            ...           ...          ...   
1059  2023-10-25  2454617609359.00  8289151342.50  365380447500  17770390600   
1060  2023-11-01  2454617609359.00  8289151342.50  365380447500  11663111300   
1061  2023-11-08  2454617609359.00  8289151342.50  365405447500  11663111300   
1062  2023-11-15  2454641775802.20  8289151342.50  365405447500  11663111300   
1063  2023-11-22  2451309459034.10  8285623147.10  365405447500  11663111300   

     tipsInflationCompensation     note

In [46]:
# Convert 'asOfDate' to datetime format
df['asOfDate'] = pd.to_datetime(df['asOfDate'])

# Add two days to the 'asOfDate' column
# df['Date'] = df['asOfDate'] + pd.to_timedelta(4, unit='D')
df['Date'] = df['asOfDate'] + pd.offsets.Week(weekday=6) 
# Convert object columns to numeric
numeric_columns = ['mbs', 'cmbs', 'tips', 'frn', 'tipsInflationCompensation', 'notesbonds', 'bills', 'agencies', 'total']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [47]:
# Create a copy of the DataFrame
SOMA_diffs = df.copy()

# Calculate differences using the .diff() function for numeric columns
SOMA_diffs[df.select_dtypes(include='number').columns] = SOMA_diffs.select_dtypes(include='number').diff()

# Create another copy of the DataFrame
SOMA_pct_diffs = df.copy()

# Calculate percent differences using the .pct_change() function for numeric columns
SOMA_pct_diffs[df.select_dtypes(include='number').columns] = SOMA_pct_diffs.select_dtypes(include='number').pct_change()


In [48]:
df.tail()

,asOfDate,mbs,cmbs,tips,frn,tipsInflationCompensation,notesbonds,bills,agencies,total,Date
1059,2023-10-25,2.454618e+12,8.289151e+09,365380447500,1.777039e+10,1.124895e+11,4183755415900,232804073900,2.347000e+09,7.264964e+12,2023-10-29
1060,2023-11-01,2.454618e+12,8.289151e+09,365380447500,1.166311e+10,1.129315e+11,4151360766500,231265518200,2.347000e+09,7.224924e+12,2023-11-05
1061,2023-11-08,2.454618e+12,8.289151e+09,365405447500,1.166311e+10,1.132194e+11,4151360766500,231265518200,2.347000e+09,7.224949e+12,2023-11-12
1062,2023-11-15,2.454642e+12,8.289151e+09,365405447500,1.166311e+10,1.134966e+11,4120788223300,231265518200,2.347000e+09,7.194400e+12,2023-11-19
1063,2023-11-22,2.451309e+12,8.285623e+09,365405447500,1.166311e+10,1.138141e+11,4120788223300,231265518200,2.347000e+09,7.191064e+12,2023-11-26


In [49]:
SOMA_diffs.tail()

,asOfDate,mbs,cmbs,tips,frn,tipsInflationCompensation,notesbonds,bills,agencies,total,Date
1059,2023-10-25,-1.672182e+10,-23207925.3,0.0,0.000000e+00,4.695342e+08,0.000000e+00,-1.726971e+09,0.0,-1.847200e+10,2023-10-29
1060,2023-11-01,0.000000e+00,0.0,0.0,-6.107279e+09,4.419936e+08,-3.239465e+10,-1.538556e+09,0.0,-4.004048e+10,2023-11-05
1061,2023-11-08,0.000000e+00,0.0,25000000.0,0.000000e+00,2.878866e+08,0.000000e+00,0.000000e+00,0.0,2.500000e+07,2023-11-12
1062,2023-11-15,2.416644e+07,0.0,0.0,0.000000e+00,2.771974e+08,-3.057254e+10,0.000000e+00,0.0,-3.054838e+10,2023-11-19
1063,2023-11-22,-3.332317e+09,-3528195.4,0.0,0.000000e+00,3.174366e+08,0.000000e+00,0.000000e+00,0.0,-3.335845e+09,2023-11-26


In [50]:
SOMA_pct_diffs.tail()

,asOfDate,mbs,cmbs,tips,frn,tipsInflationCompensation,notesbonds,bills,agencies,total,Date
1059,2023-10-25,-0.006766,-0.002792,0.000000,0.000000,0.004192,0.000000,-0.007364,0.0,-0.002536,2023-10-29
1060,2023-11-01,0.000000,0.000000,0.000000,-0.343677,0.003929,-0.007743,-0.006609,0.0,-0.005511,2023-11-05
1061,2023-11-08,0.000000,0.000000,0.000068,0.000000,0.002549,0.000000,0.000000,0.0,0.000003,2023-11-12
1062,2023-11-15,0.000010,0.000000,0.000000,0.000000,0.002448,-0.007364,0.000000,0.0,-0.004228,2023-11-19
1063,2023-11-22,-0.001358,-0.000426,0.000000,0.000000,0.002797,0.000000,0.000000,0.0,-0.000464,2023-11-26


In [51]:
import sqlite3

# Create a SQLite connection and a cursor
conn = sqlite3.connect('FED_Data.db')
cursor = conn.cursor()

# Store the original DataFrame as a table
df.to_sql('SOMA_Holdings', conn, index=False, if_exists='replace')

# Store the SOMA_diffs DataFrame as a table
SOMA_diffs.to_sql('SOMA_weekly_diff', conn, index=False, if_exists='replace')

# Store the SOMA_pct_diffs DataFrame as a table
SOMA_pct_diffs.to_sql('SOMA_weekly_percent_change', conn, index=False, if_exists='replace')

# Commit the changes and close the connection
conn.commit()
conn.close()
